# Assignment 3 : Document Retrieval and Evaluation

---



In [6]:
#import all needed libraries here

import os
from bs4 import BeautifulSoup
import lxml
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
import numpy as np
import pickle as pc
import math
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!unrar x "/content/drive/MyDrive/FIRE_Dataset_EN_2010.rar" "/content/fire_dataset/"
# !tar -xvf "/content/fire_dataset/English-Data.tgz" -C "/content"
# !cp "/content/fire_dataset/queries.txt" "/content"
# !cp "/content/fire_dataset/relevance.txt" "/content"

import shutil
shutil.unpack_archive("/content/fire_dataset/FIRE_Dataset_EN_2010/English-Data.tgz", "/content/fire_dataset/FIRE_Dataset_EN_2010/")

qrels_path="/content/drive/MyDrive/en.qrels.76-125.2010.txt"
topics_path="/content/fire_dataset/FIRE_Dataset_EN_2010/en.topics.76-125.2010.txt"


# Function to read all docs 


In [4]:
def read_doc(path):
  global docs
  global docs_list
  for file in os.listdir(path):
    if os.path.isdir(os.path.join(path,file)):
      read_doc(os.path.join(path,file))
    else:
      if file.find("index") == -1:
        fi = open(os.path.join(path,file),'r')
        bs = BeautifulSoup(fi.read())
        doc_id = bs.find("docno").text
        global i 
        i += 1
        index[doc_id] = i
        docs_list.append(doc_id)
        docs.append(bs.find("text").text)

##calling read_doc function to read whole corpus

In [26]:
global i 
index = {}
i = -1
docs = []
q_id = []
docs_list = []
read_doc("/content/fire_dataset/FIRE_Dataset_EN_2010/TELEGRAPH_UTF8")
fi = open(topics_path,'r')
data = fi.read()
bs = BeautifulSoup(data)
q_id = [int(id.text) for id in bs.find_all("num")]   # getting query id
desc = [desc.text for desc in bs.find_all("desc")]  # getting description of query
 
docs.extend(desc)
 
for id in q_id:
  i += 1
  index[id] = i

# Class for preprocessing data

In [27]:
class preprocessing:
  def __init__(self):
    self.docs_tokenized = [] # stores tokens of each doc
    self.words = set()
  def preprocess(self,docs):
    temp = []
    for doc in docs:
      doc = re.sub('(\d|[^\w\s])','',doc) #removing punctuations and digits
      doc = nltk.word_tokenize(doc.lower()) #tokenizing
      sw = set(stopwords.words('english'))
      doc = [token for token in doc  if token not in sw]# removing stop words
      lemmatizer = WordNetLemmatizer()
      doc = [lemmatizer.lemmatize(x) for x in doc] #lemmitizing
      self.docs_tokenized.append(doc)
      temp.extend(doc)
    self.words = set(temp)

In [28]:
ob1 = preprocessing() 
ob1.preprocess(docs)

# **Creating list of dictionaries where each dictionary contents word count of that document**

In [29]:
docs_dict = []    
for doc in ob1.docs_tokenized:
  temp_dict = {}
  for word in doc:
    temp_dict[word]=temp_dict.get(word,0)+1  #counting the frequency of each word
  docs_dict.append(temp_dict)    

#**Creating document frequency of each word which means how many documents have that word.**

In [30]:
df = {} #dictionary to store df
for doc in ob1.docs_tokenized:
  for word in set(doc):
    df[word] = df.get(word,0)+1  #Counting numbers of documents which contains specific word


#**Calculating inverse document frequency-IDF for each word**

In [31]:
idf = {}
N = len(docs_dict)
for word in df:
  idf[word] = math.log(N/df[word])

# **Class for finding tfidf vector**

In [32]:
class tf_idf:
  def __init__(self,doc_index,idf,docs_dict,words):
    self.doc_index=doc_index
    self.idf = idf
    self.docs_dict = docs_dict
    self.words = words
  """It returns a dictionary for document which contains word as key and it's Tf-idf score as value """  
  def getvectordict(self,doc_id):
    vector = {}
    doc = self.docs_dict[self.doc_index[doc_id]]  #Getting doc from docs dictionary with help of index
    n = sum(doc.values())
    if n!= 0:
      for word in doc:
        tf = doc[word] / n                        #Calculating tf
        tfidf = tf*self.idf[word]                 #Calculating tf-idf
        if tfidf!=0:
         vector[word] = tfidf
    return vector   


In [33]:
ob2 = tf_idf(index,idf,docs_dict,ob1.words)

#**Find tfidf vectors of queries**

In [34]:
queries_tfidf = {}
for q in q_id:
    queries_tfidf[q] = ob2.getvectordict(q)

#**Find tfidf vectors of documnets**

In [35]:
docs_tfidf = {}
for doc in docs_list:
    docs_tfidf[doc] = ob2.getvectordict(doc)

# **Calculating cosine similarity**

In [36]:
from numpy.linalg import norm
def c_s(a,b):
  dot_pr = 0
  av = list(a.values())
  bv = list(b.values())
  for word in a:
    if word in b:
      dot_pr = dot_pr + (a[word] * b[word])
  return (dot_pr/(norm(av)*norm(bv)))

In [37]:
cs = {}
for q in q_id:                    # iterating through each query
  qv = queries_tfidf[q]           # getting tfidf of specific query
  temp = {}
  for doc in docs_list:           # calculating cosine similarity of query and document
    dv = docs_tfidf[doc]    
    similarity = c_s(qv,dv)
    if  similarity != 0:               
     temp[doc] = similarity          
  cs[q] = temp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


#**Getting top 10 documents for each query on basis of there cosine similarity**

In [38]:
from heapq import nlargest
rank_list = {}
for q in q_id:
  rank_list[q] = nlargest(10,cs[q],key = cs[q].get)

# **Reading qrels**

In [39]:
fr = open(qrels_path,'r')
qr = {}
for line in fr.readlines():
  data = line.split()
  if data[0] not in qr:
   qr[data[0]] = {}
   qr[data[0]][data[2]] = int(data[3])          
  else: 
    qr[data[0]][data[2]] = int(data[3])

# **Calculating Mean Average Precision**

In [40]:
avg_precision = []                                
for q in q_id:                                             #Iterating through each query
  rk = rank_list[q]                                        # Getting rank list of specific query
  relevance = 0                                            # Store number of relevant documents found
  count = 0                                                #To store count of docs
  precision_sum = 0 
  for doc in rk: 
    count += 1                         
    if doc in qr[str(q)]:
      if qr[str(q)][doc] == 1:                             #Checking if it's relavant document 
        relevance += 1                          
        precision_sum += (relevance/count)                 #Calculating precision and adding it to local precision_sum                    
  if relevance==0:
    avg_precision.append(0)     
  else:
    avg_precision.append(precision_sum/relevance)
  
map = sum(avg_precision)/len(avg_precision)      

print("MAP : ",round(map,2))

MAP :  0.49
